# Introduction to OpenAI Agents

This notebook provides a basic introduction to working with OpenAI Agents using the OpenAI SDK.

## Setup

First, let's install and import the necessary libraries:

In [ ]:
# Uncomment to install the OpenAI library if needed
# !pip install openai python-dotenv

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Initialize the client
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

print("OpenAI client initialized successfully!")

## 1. Basic Chat Completion

Let's start with a simple chat completion:

In [ ]:
def simple_chat_completion(prompt):
    """Get a simple chat completion from OpenAI."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Test with a simple prompt
prompt = "Write a short poem about artificial intelligence."
response = simple_chat_completion(prompt)
print(response)

## 2. Using Tools with OpenAI Agents

Now, let's define some tools and see how agents can use them:

In [ ]:
import datetime

def get_current_time():
    """Tool to get the current time."""
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

def calculate(expression):
    """Tool to evaluate a mathematical expression."""
    try:
        # Using eval with caution - in production, use a safer method
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Define the tools in OpenAI's format
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current date and time",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate, e.g., '2 + 2 * 4'"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

In [ ]:
def agent_with_tools(user_message):
    """Run an agent with access to tools."""
    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to tools."},
        {"role": "user", "content": user_message}
    ]
    
    # First call to get the assistant's response
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    assistant_response = response.choices[0].message
    messages.append(assistant_response)
    
    print("\nInitial response:")
    if hasattr(assistant_response, 'content') and assistant_response.content:
        print(assistant_response.content)
    
    # Process any tool calls
    if assistant_response.tool_calls:
        print("\nTool calls requested:")
        for tool_call in assistant_response.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"- Function: {function_name}")
            print(f"  Arguments: {function_args}")
            
            # Execute the function
            if function_name == "get_current_time":
                function_response = get_current_time()
            elif function_name == "calculate":
                function_response = calculate(function_args.get("expression", ""))
            else:
                function_response = f"Error: Function {function_name} not implemented"
            
            print(f"  Result: {function_response}")
            
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            })
        
        # Get the final response after using tools
        final_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )
        
        print("\nFinal response:")
        print(final_response.choices[0].message.content)
        return final_response.choices[0].message.content
    else:
        # If no tools were used
        return assistant_response.content

In [ ]:
# Test with a question that requires tool use
agent_with_tools("What time is it now? Also, what is the result of 15 * 23 + 7?")

## 3. Multi-turn Conversation with Tools

Let's implement a multi-turn conversation that can use tools:

In [ ]:
def conversation_with_tools():
    """Run an interactive conversation with a tool-using agent."""
    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to tools. Keep your responses concise."}
    ]
    
    print("Starting conversation (type 'exit' to end)\n")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("\nAssistant: Goodbye!")
            break
        
        messages.append({"role": "user", "content": user_input})
        
        # Get assistant's response
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        assistant_message = response.choices[0].message
        messages.append(assistant_message)
        
        # Handle tool calls if any
        if assistant_message.tool_calls:
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                # Call the function
                if function_name == "get_current_time":
                    function_response = get_current_time()
                elif function_name == "calculate":
                    function_response = calculate(function_args.get("expression", ""))
                else:
                    function_response = f"Error: Unknown function {function_name}"
                
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                })
            
            # Get final response after tools
            final_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
            )
            
            print(f"\nAssistant: {final_response.choices[0].message.content}")
        else:
            print(f"\nAssistant: {assistant_message.content}")

# Uncomment to start the interactive conversation
# conversation_with_tools()

## Conclusion

This notebook has demonstrated basic interactions with OpenAI Agents, including:
- Simple chat completions
- Using tools with agents
- Multi-turn conversations

The Agents framework is powerful and allows for creating complex applications with access to external functions and data sources.